In [2]:
import pandas as pd
from pathlib import Path
import geopandas as gpd
from shapely.wkt import loads
from shapely.geometry import Point

pois_path = Path("D:/Siyu Zhao/data/Auckland region park/Auckland-parks-poi.gpkg")
pois = gpd.read_file(pois_path)

info = pd.read_csv("D:/Siyu Zhao/data/Auckland region park/infostop_with_homelocation.csv")

info["geometry"] = info["geometry"].apply(loads)
info["home_location"] = info["home_location"].apply(lambda x: loads(x) if isinstance(x, str) else None)

stay_gdf = gpd.GeoDataFrame(info, geometry="geometry", crs="EPSG:4326").to_crs(epsg=3857)
stay_gdf["home_location"] = stay_gdf["home_location"].apply(
    lambda x: gpd.GeoSeries([x], crs="EPSG:4326").to_crs(epsg=3857).iloc[0] if x else None
)

pois = pois.to_crs(epsg=3857)

# match stay point with the nearest poi
matched = gpd.sjoin_nearest(stay_gdf, pois, how="left", distance_col="poi_distance")


def get_home_distance(home_point, pois_gdf):
    if home_point is None:
        return None
    return pois_gdf.geometry.distance(home_point).min()


matched["home_distance"] = matched["home_location"].apply(lambda x: get_home_distance(x, pois))



#matched = matched[matched["distance"] <= 100]


print(matched.head())

KeyboardInterrupt: 

In [ ]:

matched.to_csv("D:/Siyu Zhao/data/Auckland region park/infostop_matched_poi.csv", index=False)